In [1]:
from typing import TypedDict
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
# from langchain_ollama.llms import OllamaLLM
import os
from dotenv import load_dotenv
import re

# Load environment variables and set OpenAI API key
load_dotenv()

ModuleNotFoundError: No module named 'langgraph'

In [10]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

In [16]:
from langchain_core.messages import AIMessage, HumanMessage, RemoveMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

workflow = StateGraph(state_schema=MessagesState)

# Define the function that calls the model
def call_model(state: MessagesState):
    system_prompt = (
        "You are an AI IELTS teacher from Smartcom, an expert in IELTS."
        "Provide your answer to user question"
        "Answer all questions to the best of your ability. "
        "The provided chat history includes a summary of the earlier conversation."
    )
    system_message = SystemMessage(content=system_prompt)
    message_history = state["messages"][:-1]  # exclude the most recent user input
    # Summarize the messages if the chat history reaches a certain size
    if len(message_history) >= 4:
        last_human_message = state["messages"][-1]
        # Invoke the model to generate conversation summary
        summary_prompt = (
            "Distill the above chat messages into a single summary message. "
            "Include as many specific details as you can."
        )
        summary_message = model.invoke(
            message_history + [HumanMessage(content=summary_prompt)]
        )

        # Delete messages that we no longer want to show up
        delete_messages = [RemoveMessage(id=m.id) for m in state["messages"]]
        # Re-add user message
        human_message = HumanMessage(content=last_human_message.content)
        # Call the model with summary & response
        response = model.invoke([system_message, summary_message, human_message])
        message_updates = [summary_message, human_message, response] + delete_messages
    else:
        message_updates = model.invoke([system_message] + state["messages"])

    return {"messages": message_updates}

# Define the node and edge
workflow.add_node("model", call_model)
workflow.add_edge(START, "model")

# Add simple in-memory checkpointer
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [25]:
from typing import TypedDict, Dict, List, Optional

demo_ephemeral_chat_history = [
    HumanMessage(content="Hey there! I'm Nemo."),
    AIMessage(content="Hello!"),
    HumanMessage(content="How are you today?"),
    AIMessage(content="Fine thanks!"),
]

demo_chat_history = [
    {"human": "Hey there! I'm Nemo.", "ai": "Hello!"},
    {"human": "How are you today?", "ai": "Fine thanks!"},
]

def _format_chat_history(chat_history: List[Dict[str, str]]) -> List:
    converted_chat_history = []
    for message in chat_history:
        if message.get("human") is not None:
            converted_chat_history.append(HumanMessage(content=message["human"]))
        if message.get("ai") is not None:
            converted_chat_history.append(AIMessage(content=message["ai"]))
    return converted_chat_history

app.invoke(
    {
        "messages": demo_ephemeral_chat_history
        + [HumanMessage(content="What's my name?")]
    },
    config={"configurable": {"thread_id": "2"}},
)

{'messages': [AIMessage(content='In the chat, Nemo introduced himself and asked how I was doing. I replied that I was fine, and he confirmed his name as Nemo.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 110, 'total_tokens': 140, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'stop', 'logprobs': None}, id='run-407d156a-5140-47a0-8759-52d6b5e4459a-0', usage_metadata={'input_tokens': 110, 'output_tokens': 30, 'total_tokens': 140, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
  HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}, id='8b54d576-e331-44f0-8dfc-06fd962097d9'),
  

In [22]:
x = _format_chat_history(demo_chat_history)
x

[HumanMessage(content="Hey there! I'm Nemo.", additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hello!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='How are you today?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Fine thanks!', additional_kwargs={}, response_metadata={})]

In [24]:
app.invoke(
    {
        "messages": x
        + [HumanMessage(content="What's my name?")]
    },
    config={"configurable": {"thread_id": "2"}},
)

{'messages': [AIMessage(content='In the chat, Nemo introduced himself and asked how I was doing. I responded that I was fine, and Nemo confirmed his name as Nemo.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 108, 'total_tokens': 138, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'stop', 'logprobs': None}, id='run-c58e0b56-efac-4c07-b214-76cef1f4d8ab-0', usage_metadata={'input_tokens': 108, 'output_tokens': 30, 'total_tokens': 138, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
  HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}, id='8fc5fe65-3243-42ab-9e7a-82fb97d4957e')

In [19]:
demo_ephemeral_chat_history

[HumanMessage(content="Hey there! I'm Nemo.", additional_kwargs={}, response_metadata={}, id='3218740f-dfcc-40dd-9c4c-7aafe95343df'),
 AIMessage(content='Hello!', additional_kwargs={}, response_metadata={}, id='8d136e8f-9fb3-4e78-90e3-dd66807ff026'),
 HumanMessage(content='How are you today?', additional_kwargs={}, response_metadata={}, id='b6cc3ece-cebb-4e12-b2e5-fcc3337d4911'),
 AIMessage(content='Fine thanks!', additional_kwargs={}, response_metadata={}, id='1593578a-d392-4b4e-b164-be95f0888ae2')]